# 지난 시간 복습

1. Backpropagation-Theory
2. Manual Backpropagation
3. Automatic Backpropagation  
--- 
현재 함수의 미분값을 $cur.gy$, 현재 함수의 도함수, 현재 함수의 입력, 이전 함수의 미분값을 각각 $cur', cur.input, prev.gy$라고 한다.  
$cur.gy = prev.gy * cur'(cur.input)$

$x \rightarrow A(x) \rightarrow B(a) \rightarrow C(b) \rightarrow y$ (Forward)  

$dx \leftarrow A'(x) \leftarrow B'(a) \leftarrow C'(b) \leftarrow dy$ (Backward)


In [52]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data
        self.creator = None
        self.dx = None

    def backward(self):
        func = self.creator
        if func is not None:
            x = func.input
            dx = func.backward(x.data)
            prev_dx = self.dx
            x.dx = dx * prev_dx
            x.backward()
        return self.dx
    
    def set_creator(self, func):
        self.creator = func

class Function:
    def __call__(self, var):
        x = var.data
        y = self.forward(x)
        y = Variable(y)

        y.set_creator(self)
        self.input = var
        self.output = y

        return y
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, x):
        return np.exp(x)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, x):
        return 2 * x

In [53]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.dx = np.array(1.0)
y.backward()
print(x.dx)

3.297442541400256


# 이번주에는 총 4가지를 배운다
1. Backward 연산을 재귀 => 반복
2. Function subclass의 사용을 편리하게
3. Code 작동 Test 기법: UnitTest module
4. 입력 크기 확장

# Not Clone coding

In [58]:
import numpy as np

class Variable:
    def __init__(self, data):
        if isinstance(data, np.ndarray) or isinstance(data, np.float_) or isinstance(data, np.int_):
            self.data = data
        else:
            raise Exception("지원하지 않는 데이터 형식입니다.")
        self.creator = None
        self.dx = None

    def backward(self):
        funcs = []
        funcs.append(self.creator)
        while funcs:
            func = funcs.pop()
            x = func.input
            dx = func.backward(x.data)
            if self.dx is None:
                self.dx = np.array(1.0)
            prev_dx = func.output.dx
            x.dx = dx * prev_dx
            if x.creator is not None:
                funcs.append(x.creator)
        return self.dx
    
    def set_creator(self, func):
        self.creator = func

class Function:
    def __call__(self, var):
        x = var.data
        y = self.forward(x)
        y = Variable(y)

        y.set_creator(self)
        self.input = var
        self.output = y

        return y
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, x):
        return np.exp(x)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, x):
        return 2 * x
    
def exp(x=None):
    if x is not None:
        return Exp()(x)
    else:
        return Exp()

def square(x=None):
    if x is not None:
        return Square()(x)
    else:
        return Square()

In [43]:
a = np.array(1.0)
b = np.array([1.0])
c = np.array([2.0])
d = b * c
e = a * b
f = np.array(2.0)


print(type(a), a.ndim)
print(type(b), b.ndim)
print(type(c), c.ndim)
print(type(d), d.ndim)
print(type(e), e.ndim)
print(type(a*f), (a*f).ndim)
print(a*f)
print(d)
print(a)
print((a*f)*c)
if isinstance((a*f), np.float_):
    print(1)

<class 'numpy.ndarray'> 0
<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'> 1
<class 'numpy.float64'> 0
2.0
[2.]
1.0
[4.]
1


In [59]:
x = Variable(np.array(0.5))

y = square(exp(square(x)))
y.backward()
x.dx

3.297442541400256

# 1. Backward: Recurrence ==> Iteration

In [60]:
class Variable(object):
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function(object):
    def __call__(self, var):
        x = var.data
        y = self.forward(x)
        y = Variable(y)
        y.set_creator(self)
        self.input = var
        self.output = y
        return y
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [61]:

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 2. Function subclass의 사용을 편리하게

In [70]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable(object):
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function(object):
    def __call__(self, var):
        x = var.data
        y = as_array(self.forward(x))
        y = Variable(y)
        y.set_creator(self)
        self.input = var
        self.output = y
        return y
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [71]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
#y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 3. Code 작동 Test 기법: UnitTest module

In [95]:
import unittest

def numerical_diff(f, x, eps=1e-4):
    x1 = Variable(x.data - eps)
    x2 = Variable(x.data + eps)
    y1 = f(x1)
    y2 = f(x2)
    return (y2.data - y1.data) / (2 * eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [96]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest.test_backward) ... ok
test_forward (__main__.SquareTest.test_forward) ... ok
test_gradient_check (__main__.SquareTest.test_gradient_check) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK


# 4. 입력 크기 확장

In [103]:
class Function(object):
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )

In [104]:
xs = [Variable(np.array(2.0)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5.0
